# `nepare` ASAP `polaris` Competition

This notebook demonstrates using Neural Pairwise Regression (via `nepare`) with the `polaris` benchmarking library to compete in the ASAP discovery competition.

See the `meta` directory in the `nepare` repository for some more theoretical exploration of what this notebook does.

## Requirements
This should work on Python 3.10 or newer - I ran with 3.12

The following dependencies are required:
 - polaris-lib >=0.11.6
 - pandas
 - rdkit
 - lightning
 - torch
 - fastprop
 - mordredcommunity
 - chemprop ~2.1
 - ipywidgets

You will also need to run `pip install .` in the repository's root directory to install `nepare`.

## `polaris` Setup

After running `polaris login` on the command line, we can import everything (checking that the version is recent enough) and then download the benchmark data.

In [1]:
import polaris as po
import pandas as pd

In [2]:
%%capture
competition = po.load_competition("asap-discovery/antiviral-potency-2025")
# or
# competition = po.load_competition("asap-discovery/antiviral-admet-2025")

In [3]:
train, test = competition.get_train_test_split()
test_df: pd.DataFrame = test.as_dataframe()
train_df: pd.DataFrame = train.as_dataframe()

In [4]:
train_df

,CXSMILES,pIC50 (SARS-CoV-2 Mpro),pIC50 (MERS-CoV Mpro)
0,COC[C@]1(C)C(=O)N(C2=CN=CC3=CC=CC=C23)C(=O)N1C...,NaN,4.19
1,C=C(CN1CCC2=C(C=C(Cl)C=C2)C1C(=O)NC1=CN=CC2=CC...,5.29,4.92
2,CNC(=O)CN1C[C@]2(C[C@H](C)N(C3=CN=CC=C3C3CC3)C...,NaN,4.73
3,C=C(CN1CCC2=C(C=C(Cl)C=C2)C1C(=O)NC1=CN=CC2=CC...,6.11,4.90
4,C=C(CN1CCC2=C(C=C(Cl)C=C2)C1C(=O)NC1=CN=CC2=CC...,5.62,4.81
...,...,...,...
1026,CNS(=O)(=O)OCC(=O)N1CCN(CC2=CC=CC(Cl)=C2)[C@H]...,6.38,5.57
1027,O=C(CC1=CN=CC2=CC=CC=C12)N1CC[C@@H]2CCCC[C@H]2...,6.09,4.60
1028,CNC(=O)[C@H]1CCCN(C(=O)CC2=CN=CC3=CC=CC=C23)C1...,NaN,4.22
1029,C[C@H]1CCCN(C(=O)CC2=CN=CC3=CC=CC=C23)[C@H]1C ...,5.06,4.40


## Model Setup
We can pick a number of different methods for representing molecules as a vector.
For this notebook we will consider just two: `mordred` and ChemProp.
The former uses a vector of ~1,600 molecular descriptors which can be calculated from the graph.
This embedding is _fixed_ or _static_, i.e. it does not change during training.
This has the advantage of packing a lot of chemical knowledge into the representation, but anything that we miss can't be added during training.
ChemProp on the other hand is a _learned_ embedding, which will adapt during training to best regress our target.

There is also the combination of the two - learn a representation during training, and then simply concatenate the molecular descriptors to said representation.
In _theory_ this should cover the advantages of the two approaches, so we'll use it here.

### Define an Embedding Module for `nepare`
We can pass a custom-built module for learning an embedding during training as shown below:

In [5]:
from typing import OrderedDict

import torch

class MordredChempropEmbedder(torch.nn.Module):
    def __init__(self, mp, agg, mordred_size, mordred_layers):
        super().__init__()
        # chemprop
        self.mp = mp
        self.agg = agg

        # mordred
        _modules = OrderedDict()
        _modules["feature_dropout"] = torch.nn.Dropout(0.80)
        activation = torch.nn.ReLU
        for i in range(mordred_layers):
            _modules[f"hidden_{i}"] = torch.nn.Linear(1_613 if i == 0 else mordred_size, mordred_size)
            if i < (mordred_layers - 1):  # skip on last
                _modules[f"{activation.__name__.lower()}_{i}"] = activation()
                _modules[f"dropout_{i}"] = torch.nn.Dropout(0.5)
        self.fnn = torch.nn.Sequential(_modules)

        # shared
        self.bn = torch.nn.BatchNorm1d(mp.output_dim + mordred_size)
        self.dropout = torch.nn.Dropout(0.5)
        self.relu = activation()

    def forward(self, batch):
        bmg, feats = batch
        H = self.mp(bmg)
        Z = self.agg(H, bmg.batch)
        f = self.fnn(feats)
        return self.relu(self.dropout(self.bn(torch.cat((Z, f), dim=1))))

A note on the architecture - we aggressively regularize the `mordred` features to prevent the model using only those instead of ChemProp.

### Identifying Validation Set
My experience so far shows that these pairwise difference learning networks tend to overfit _very_ easily (thus all the regularization).
We need to use early stopping to deal with this, and since we know what area of chemical space we want to extrapolate into we can also choose our validation set to reflect this. We will also downsample the data (again, overfitting) during this process.

In [6]:
from random import Random

from rdkit.Chem import DataStructs, rdFingerprintGenerator
from rdkit import Chem

This is just a convenience function that looks like the old RDKit fingerprint generation syntax (I prefer the old way `¯\_(ツ)_/¯`).

In [7]:
def _smi2fp(smi):
    fpg = rdFingerprintGenerator.GetMorganGenerator(radius=4)
    return fpg.GetFingerprint(Chem.MolFromSmiles(smi))

We want our validation set to tell us to stop training once we start failing to extrapolate to the new chemical space in the test set that we care about.
At the same time, we want to train on that chemical so that we can improve model performance.

To that end, we have this `split` function, which selects a training set of equal size to the test set such that the most similar molecule in the available data to each test point is present (or the _n_-th most similar, if the most has already been selected) in training.
We then randomly select some of the un-selected points for validation to control early stopping.
During validation all validation points are anchored against the training data, so this will give us a good idea of when the chemical space of the validation set has been well fit.

In [8]:
def split(train_smis, test_smis):
    train_fps = list(map(_smi2fp, train_smis))
    test_fps = list(map(_smi2fp, test_smis))

    train_idxs = set()  # specifically use a set here to avoid duplicates
    for fp in test_fps:
        sims = [(i, DataStructs.FingerprintSimilarity(fp, _fp)) for i, _fp in enumerate(train_fps)]
        while 1:  # continue selecting the most similar molecule until one is found which is not already selected
            most_similar_idx = max(sims, key=lambda t: t[1])[0]
            if most_similar_idx in train_idxs:
                sims[most_similar_idx] = (most_similar_idx, 0.0)
            else:
                train_idxs.add(most_similar_idx)
                break
    train_idxs = list(train_idxs)

    rng = Random(42)
    rng.shuffle(train_idxs)
    _n = int(len(train_idxs) * 0.05)
    val_idxs = train_idxs[:_n]
    train_idxs = train_idxs[_n:]

    return train_idxs, val_idxs

### Helpers
Finally, before we start training, we will define some functions and classes for dataloading, featurization, etc.

Most of these aren't commented, but they generally only wrap other well-documented code or do something obvious.

In [9]:
from rdkit.Chem import MolFromSmiles
from chemprop.featurizers import MolGraphCache, SimpleMoleculeMolGraphFeaturizer

map `list[SMILES] -> list[chemprop MolGraphs]`
we use `MolGraphCache` here (so that one could easily substitute `MolGraphCacheOnTheFly`) but we would instead just ust `mgc = list(map(featurizer, mols))` in place of this function.

In [10]:
def smiles2molgraphcache(smiles: list[str]):
    mols = list(map(MolFromSmiles, smiles))
    featurizer = SimpleMoleculeMolGraphFeaturizer()
    mgc = MolGraphCache(mols, [None] * len(mols), [None] * len(mols), featurizer)
    return mgc

In [11]:
from mordred import Calculator, descriptors
from fastprop.data import standard_scale
import numpy as np

map `list[SMILES]` -> imputed, scaled, winsorized features tensor

In [12]:
def smi2features(smis, feature_means=None, feature_vars=None):
    calc = Calculator(descriptors, ignore_3D=True)
    train_features = calc.pandas(map(MolFromSmiles, smis), nmols=len(smis), quiet=True).fill_missing()
    X = torch.tensor(train_features.to_numpy(dtype=np.float32), dtype=torch.float32)
    if feature_means is None or feature_vars is None:
        X, feature_means, feature_vars = standard_scale(X)
    else:
        X = standard_scale(X, feature_means, feature_vars)
    X.clamp_(-3, 3)
    return X, feature_means, feature_vars

In [13]:
from typing import Iterable

from chemprop.data import MolGraph, BatchMolGraph

In [14]:
def _collate_fn(batch: Iterable[tuple[tuple[MolGraph, torch.Tensor], tuple[MolGraph, torch.Tensor], float]]):
    mgs_1, feats_1, mgs_2, feats_2, ys = [], [], [], [], []
    for item in batch:
        mgs_1.append(item[0][0])
        feats_1.append(item[0][1])
        mgs_2.append(item[1][0])
        feats_2.append(item[1][1])
        ys.append(item[2])
    return ((BatchMolGraph(mgs_1), torch.stack(feats_1, dim=0)), (BatchMolGraph(mgs_2), torch.stack(feats_2, dim=0)), torch.tensor(np.array(ys), dtype=torch.float32))


In [15]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr

In [16]:
def evaluate_predictions(y_true, y_pred):
    return {"Pearson r": pearsonr(y_true, y_pred).correlation, "MSE": mean_squared_error(y_true, y_pred), "MAE": mean_absolute_error(y_true, y_pred)}

## Train Per-Task Models
`nepare` works by taking the difference in a target value for each input.
Unfortunately this means that when target values are _missing_, we can't simply mask single inputs - we need to mask entire pairs.
Rather than do this and end up with a really sparse training set, we will just train one model for each target property.

This is OK in terms of the amount of training data we have - we might drop 25% of the data, but we end up squaring the amount of remaining data, nullifying the issue.

In [17]:
from pathlib import Path

import lightning
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks.model_checkpoint import ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger
from chemprop.nn.agg import MeanAggregation
from chemprop.nn.message_passing import BondMessagePassing
from fastprop.data import standard_scale, inverse_standard_scale

from nepare.data import PairwiseAugmentedDataset, PairwiseAnchoredDataset, PairwiseInferenceDataset
from nepare.nn import LearnedEmbeddingNeuralPairwiseRegressor
from nepare.inference import predict

This huge block of code is responsible for training each model, reporting its performance, and running inference on the test data.
Check the inline comments for more explanation.

In [18]:
predictions = {}  # for polaris
val_score = []  # for my own checking of model performance
output_dir = Path("lightning_logs")
tasks = list(competition.target_cols)
for task_n, task_name in enumerate(tasks):
    # copy from the train dataframe each time to avoid accidentally messing up all the training data
    task_df = train_df[["CXSMILES", task_name]].copy()
    task_df.dropna(inplace=True)
    task_df.reset_index(inplace=True)

    train_idxs, val_idxs = split(task_df["CXSMILES"], test_df["CXSMILES"])

    # rescale our target values, both for metric reporting and NN performance
    train_targets = torch.tensor(task_df[task_name].iloc[train_idxs].to_numpy(), dtype=torch.float32).reshape(-1, 1)  # 2d!
    train_targets, target_means, target_vars = standard_scale(train_targets)
    val_targets = torch.tensor(task_df[task_name].iloc[val_idxs].to_numpy(), dtype=torch.float32).reshape(-1, 1)  # 2d!
    val_targets = standard_scale(val_targets, target_means, target_vars)

    # featurize the data
    train_features, feature_means, feature_vars = smi2features(task_df["CXSMILES"][train_idxs])
    val_features, _, _ = smi2features(task_df["CXSMILES"][val_idxs], feature_means, feature_vars)
    test_features, _, _ = smi2features(test_df["CXSMILES"], feature_means, feature_vars)
    train_mgc = smiles2molgraphcache(task_df["CXSMILES"][train_idxs])
    val_mgc = smiles2molgraphcache(task_df["CXSMILES"][val_idxs])
    test_mgc = smiles2molgraphcache(test_df["CXSMILES"])
    train_tuples = [(train_mgc[i], train_features[i, :]) for i in range(len(train_targets))]
    val_tuples = [(val_mgc[i], val_features[i, :]) for i in range(len(val_targets))]
    test_tuples = [(test_mgc[i], test_features[i, :]) for i in range(len(test_mgc))]
    # setup the datasets
    train_dataset = PairwiseAugmentedDataset(train_tuples, train_targets, how='sut')
    val_dataset = PairwiseAnchoredDataset(train_tuples, train_targets, val_tuples, val_targets, how='half')
    val_absolute_dataset = PairwiseInferenceDataset(train_tuples, train_targets, val_tuples, how='half')
    test_dataset = PairwiseInferenceDataset(train_tuples, train_targets, test_tuples, how='half')
    # build the model
    _size = 64
    _layers = 3
    mp = BondMessagePassing(d_h=_size, depth=_layers)
    agg = MeanAggregation()
    embedder = MordredChempropEmbedder(mp, agg, _size, _layers)
    npr = LearnedEmbeddingNeuralPairwiseRegressor(embedder, 2*_size, 2*_size, _layers, lr=1e-4)

    # classic lightning training, inference
    loader_kwargs = dict(batch_size=256, collate_fn=_collate_fn, persistent_workers=True, num_workers=1)
    train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, **loader_kwargs)
    val_loader = torch.utils.data.DataLoader(val_dataset, **loader_kwargs)
    val_absolute_loader = torch.utils.data.DataLoader(val_absolute_dataset, **loader_kwargs)
    predict_loader = torch.utils.data.DataLoader(test_dataset, **loader_kwargs)
    early_stopping = EarlyStopping(monitor="validation/loss", patience=10)
    name = "".join(c if c.isalnum() else "_" for c in task_name)
    model_checkpoint = ModelCheckpoint(dirpath=output_dir / name, monitor="validation/loss")
    logger = TensorBoardLogger(save_dir=output_dir, name=name, default_hp_metric=False)
    trainer = lightning.Trainer(max_epochs=100, log_every_n_steps=1, callbacks=[early_stopping, model_checkpoint], logger=logger)
    print(npr)
    trainer.fit(npr, train_loader, val_loader)
    npr = LearnedEmbeddingNeuralPairwiseRegressor.load_from_checkpoint(model_checkpoint.best_model_path)

    # we use the predict function from nepare, which automatically maps difference-space predictions back to absolute space
    y_pred, y_stdev = predict(npr, val_absolute_loader)

    # checking our performance on the validation set
    # leave in the scaled space so that the average is weighted equally among the targets
    results_dict = evaluate_predictions(val_targets.flatten().numpy(), y_pred.flatten().numpy())
    val_score.append((len(val_targets), results_dict["MAE"]))
    # now go back to correct scale
    y_pred = inverse_standard_scale(y_pred, target_means, target_vars)
    y_true = inverse_standard_scale(val_targets, target_means, target_vars)
    print(f"{task_name=}:\n - {"\n - ".join([f'{name}: {score:.4f}' for name, score in evaluate_predictions(y_true.flatten().numpy(), y_pred.flatten().numpy()).items()])}")

    # finally predict on the test set, descale, and save them for later
    y_pred, y_stdev = predict(npr, predict_loader)
    y_pred = inverse_standard_scale(y_pred, target_means, target_vars)
    predictions[task_name] = y_pred.flatten().tolist()

/home/jackson/neural-pairwise-regression/.venv/lib/python3.12/site-packages/mordred/_base/pandas_module.py:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  t[t.applymap(is_missing)] = value
/home/jackson/neural-pairwise-regression/.venv/lib/python3.12/site-packages/mordred/_base/pandas_module.py:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  t[t.applymap(is_missing)] = value
/home/jackson/neural-pairwise-regression/.venv/lib/python3.12/site-packages/mordred/_base/pandas_module.py:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  t[t.applymap(is_missing)] = value
/home/jackson/neural-pairwise-regression/.venv/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'embedding_module' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['embedding_module

LearnedEmbeddingNeuralPairwiseRegressor(
  (fnn): Sequential(
    (hidden_0): Linear(in_features=256, out_features=128, bias=True)
    (relu_0): ReLU()
    (hidden_1): Linear(in_features=128, out_features=128, bias=True)
    (relu_1): ReLU()
    (hidden_2): Linear(in_features=128, out_features=128, bias=True)
    (relu_2): ReLU()
    (readout): Linear(in_features=128, out_features=1, bias=True)
  )
  (embedding_module): MordredChempropEmbedder(
    (mp): BondMessagePassing(
      (W_i): Linear(in_features=86, out_features=64, bias=False)
      (W_h): Linear(in_features=64, out_features=64, bias=False)
      (W_o): Linear(in_features=136, out_features=64, bias=True)
      (dropout): Dropout(p=0.0, inplace=False)
      (tau): ReLU()
      (V_d_transform): Identity()
      (graph_transform): Identity()
    )
    (agg): MeanAggregation()
    (fnn): Sequential(
      (feature_dropout): Dropout(p=0.8, inplace=False)
      (hidden_0): Linear(in_features=1613, out_features=64, bias=True)
     

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


task_name='pIC50 (MERS-CoV Mpro)':
 - Pearson r: 0.7336
 - MSE: 0.3083
 - MAE: 0.4298


Predicting: |          | 0/? [00:00<?, ?it/s]

The hope is that the below number roughly matches with the actual performance on the `polaris` leaderboard.
I say _hope_ because the composition of the blind test set and my validation set will invariably be different, just hopefully not _so_ different that the validation set actually causes _bad_ early stopping (i.e. too early or too late).

In [19]:
print(f"Validation Scaled Weighted Average MAE {sum(n*s for n, s in val_score) / sum(n for n, _ in val_score):.4f}")

Validation Scaled Weighted Average MAE 0.3680


This last block is commented out because it will fail (unless you are me) - you can replace the inputs with your own if you are submitting this for yourself.

In [21]:
competition.submit_predictions(
    predictions=predictions,
    prediction_name="nepare",
    prediction_owner="jacksonburns",
    report_url="https://github.com/JacksonBurns/neural-pairwise-regression/blob/main/meta",
    github_url = "https://github.com/JacksonBurns/neural-pairwise-regression/blob/main/notebooks/polaris_asap.ipynb",
    description = "Neural Pairwise Regression with ChemProp + Mordred",
)

Output()

[17:37:31]  Success: Submitting competition predictions                                               ]8;id=752469;file:///home/jackson/neural-pairwise-regression/.venv/lib/python3.12/site-packages/polaris/utils/context.py\context.py]8;;\:]8;id=564660;file:///home/jackson/neural-pairwise-regression/.venv/lib/python3.12/site-packages/polaris/utils/context.py#53\53]8;;\